### Objective:
To classify images in the Caltech-256 dataset, which is an improvement over Caltech-101 dataset using a Convolutional Neural Network.

### Problem Statement
To build and implement a Convolutional Neural Network model to classify images in the Caltech-256 dataset.

At the end of this competition, you will be able to:

* Load and extract features of images available in the Caltech-256 dataset using ImageDataGenerator

* Build convolutional neural networks using either Keras or PyTorch deep learning libraries

* Use the pre-trained models using either Keras or PyTorch deep learning libraries

### Description:
Caltech-256 is an object recognition dataset containing approximately 30,000 real-world images, of different sizes, spanning 256 classes (256 object classes and an additional clutter class). Each class is represented by at least 80 images. The dataset is a superset of the Caltech-101 dataset.

Here is a handy link to Kaggle's competition documentation (https://www.kaggle.com/docs/competitions), which includes, among other things, instructions on submitting predictions (https://www.kaggle.com/docs/competitions#making-a-submission).

### Instructions for downloading train and test data are as follows:

### 1. Create an API key in Kaggle.

To do this, go to the competition site on Kaggle at https://www.kaggle.com/t/185418aa7ed24db3b98ed851a4db2b41 and click on user then click on your profile as shown below. Click Account.

![alt text](https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/Capture-NLP.PNG)



### 2. Next, scroll down to the API access section and click on **Create New Token** to download an API key (kaggle.json).

![alt text](https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/Capture-NLP_1.PNG)

### 3. Upload your kaggle.json file using the following snippet in a code cell:



In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"abhaykumardnnai","key":"36e0389fab17e1404c7fa2007ae4578b"}'}

In [2]:
#If successfully uploaded in the above step, the 'ls' command here should display the kaggle.json file.
%ls

CalTech-256-GoogleNet_v23_adv.pth  kaggle.json  sample_data/


### 4. Install the Kaggle API using the following command


In [3]:
# !pip install -U -q kaggle==1.5.8

### 5. Move the kaggle.json file into ~/.kaggle, which is where the API client expects your token to be located:



In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
#Execute the following command to verify whether the kaggle.json is stored in the appropriate location: ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [6]:
!chmod 600 /root/.kaggle/kaggle.json #run this command to ensure your Kaggle API token is secure on colab

### 6. Now download the Test Data from Kaggle

**NOTE: If you get a '403 - Not Found' error after running the cell below, it is most likely that the user (whose kaggle.json is uploaded above) has not 'accepted' the rules of the competition and therefore has 'not joined' the competition.**

If you encounter **401-unauthorised** download latest **kaggle.json** by repeating steps 1 & 2

In [7]:
#If you get a forbidden link, you have most likely not joined the competition.
!kaggle competitions download -c classification-of-caltech-256-images

 97% 305M/316M [00:02<00:00, 111MB/s]
100% 316M/316M [00:03<00:00, 108MB/s]


In [8]:
!mkdir /content/Kaggle2Test
!unzip classification-of-caltech-256-images -d /content/Kaggle2Test/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Kaggle2Test/test/4759.jpg  
  inflating: /content/Kaggle2Test/test/476.jpg  
  inflating: /content/Kaggle2Test/test/4760.jpg  
  inflating: /content/Kaggle2Test/test/4761.jpg  
  inflating: /content/Kaggle2Test/test/4762.jpg  
  inflating: /content/Kaggle2Test/test/4763.jpg  
  inflating: /content/Kaggle2Test/test/4764.jpg  
  inflating: /content/Kaggle2Test/test/4765.jpg  
  inflating: /content/Kaggle2Test/test/4766.jpg  
  inflating: /content/Kaggle2Test/test/4767.jpg  
  inflating: /content/Kaggle2Test/test/4768.jpg  
  inflating: /content/Kaggle2Test/test/4769.jpg  
  inflating: /content/Kaggle2Test/test/477.jpg  
  inflating: /content/Kaggle2Test/test/4770.jpg  
  inflating: /content/Kaggle2Test/test/4771.jpg  
  inflating: /content/Kaggle2Test/test/4772.jpg  
  inflating: /content/Kaggle2Test/test/4773.jpg  
  inflating: /content/Kaggle2Test/test/4774.jpg  
  inflating: /content/Kaggle2Test/test/4775.jpg  
 

In [9]:
# !mkdir Kaggle2Test/test
# !mv test/* Kaggle2Test/test/

### 7. Download the Train Data

In [10]:
%%capture
!wget https://cdn.iisc.talentsprint.com/DLFA/Experiment_related_data/Caltech_256_Train.zip

!unzip "Caltech_256_Train.zip"

## Grading = 10 Marks

## YOUR CODING STARTS FROM HERE

### Import Required packages

In [11]:
# import the libraries
import numpy as np
import pandas as pd
import os,shutil,glob,PIL
import pathlib
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm
from collections import defaultdict
from collections import Counter

### **Stage 1:** Data Loading and preprocessing of Images (3 points)

#### Analyze the shape of images and distribution of classes

**The below two cells were run with transform without normalization to get appropriate mean and standard deviation.**

In [12]:
# mean = 0.0
# for img, _ in train_data:
#     mean += img.mean([1,2])
# mean = mean/len(train_data)
# print(mean)

In [13]:
# sumel = 0.0
# countel = 0
# for img, _ in train_data:
#     img = (img - mean.unsqueeze(1).unsqueeze(1))**2
#     sumel += img.sum([1, 2])
#     countel += torch.numel(img[0])
# std = torch.sqrt(sumel/countel)
# print(std)

In [14]:
# Normalize with mean and std
# train_transform = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor(), transforms.Normalize((0.4839, 0.4528, 0.3962), (0.2702, 0.2655, 0.2745))])
# train_transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
# transform = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor(), transforms.Normalize((0.5503, 0.5315, 0.5028), (0.3162, 0.3125, 0.3263))])

In [15]:
# Define train and test transforms for data preprocessing and image augmentation
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

**We will be adjusting these values 224 224 after calculating mean and std dev. later on.**

In [16]:
# Loading the train set file
train_data_folder = "/content/Caltech_256_Train" # Train directory for loading images
total_dataset = datasets.ImageFolder(root=train_data_folder, transform=train_transform)
# train_data = datasets.ImageFolder(root=train_data_folder)

In [17]:
# splitting between train and Validation set
train_size = int(0.9 * len(total_dataset))  # 90% for training
val_size = len(total_dataset) - train_size  # Remaining 10% for validation
train_data, val_data = torch.utils.data.random_split(total_dataset, [train_size, val_size])

In [18]:
#defiining train_batch_size
train_batch_size = 64

In [19]:
# Create data loaders for training and validation
train_loader = torch.utils.data.DataLoader(train_data, batch_size=train_batch_size,shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=train_batch_size,shuffle=False)

In [20]:
# Loading the test set file
test_data_folder = "/content/Kaggle2Test" # Test directory for loading images
test_data = datasets.ImageFolder(root=test_data_folder, transform=test_transform)

In [21]:
test_data.classes

['test']

In [22]:
# # Initializing batch size
# batch_size = 64

# # Loading the train dataset
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [23]:
# Generate a batches of images and labels
train_images, train_labels = next(iter(train_loader))
train_images.shape, train_labels.shape

(torch.Size([64, 3, 224, 224]), torch.Size([64]))

In [24]:
# labels Translator
label_names = {v: k for k, v in total_dataset.class_to_idx.items()}
# label_names

In [25]:
# Create a grid of images along with their corresponding labels
# L = 3
# W = 3

# fig, axes = plt.subplots(L, W, figsize = (10, 10))
# axes = axes.reshape(-1)

# for i in np.arange(0, L*W):
#     axes[i].imshow(train_images[i].permute(1, 2, 0))
#     axes[i].set_title(label_names[train_labels[i].item()])
#     axes[i].axis('on')

# plt.tight_layout()

In [26]:
# mean = 0.0
# for img, _ in train_data:
#     mean += img.mean([1,2])
# mean = mean/len(train_data)
# print(mean)

In [27]:
# sumel = 0.0
# countel = 0
# for img, _ in train_data:
#     img = (img - mean.unsqueeze(1).unsqueeze(1))**2
#     sumel += img.sum([1, 2])
#     countel += torch.numel(img[0])
# std = torch.sqrt(sumel/countel)
# print(std)

In [28]:
# sum = 0
# for label in train_data.classes:
#     num_img = len(train_data.targets[train_data.targets == train_data.class_to_idx[label]])
#     print (num_img)
# print (sum)

num_classes = len(total_dataset.classes)
dataset_size = len(total_dataset)
classes = total_dataset.classes
img_dict = {}
for i in range(num_classes):
    img_dict[classes[i]] = 0

for i in range(dataset_size):
    img, label = total_dataset[i]
    img_dict[classes[label]] += 1

# img_dict

**Our observation is all those minority classes we need to go for augmenting new samples.**

In [29]:
# No of Categories
print(len(total_dataset.classes))

256


In [30]:
# Number of training samples
print(len(total_dataset))

21008


In [31]:
# Size of one training image
print(train_data[0][0].size(), val_data[0][0].size())

torch.Size([3, 224, 224]) torch.Size([3, 224, 224])


In [32]:
max(img_dict,key=img_dict.get)

'Clutter'

In [33]:
# print(img_dict.values())
# total = sum(img_dict.values())

In [34]:
# To test whether GPU instance is present in the system of not.
use_cuda = torch.cuda.is_available()
print('Using PyTorch version:', torch.__version__, 'CUDA:', use_cuda)

Using PyTorch version: 2.0.1+cu118 CUDA: True


In [35]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [ ]:
model_ft = models.googlenet(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes)
#model.softmax = nn.softmax()
model_ft = model_ft.to(device)
# print(model_ft)

**Skip the below step if you donot wish to do incremental training and move to the next step. This below code will perform incremental training on your loaded model. Please ensure you have uploaded the same model to colab before running this**

In [37]:
model_ft = models.googlenet(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,num_classes)
PATH='/content/CalTech-256-GoogleNet_v23_adv.pth'
model_ft.load_state_dict(torch.load(PATH))
model_ft = model_ft.to(device)
print(model_ft)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 175MB/s]


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [38]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(), lr=0.001)

In [39]:
epsilon = 0.005  # Perturbation strength for FGSM

In [40]:
train_accu = []     # Empty list for saving train accuracy
train_losses = []   # Empty list for saving train losses
val_accu = []

In [41]:
def train(epoch):
  print('\nEpoch : %d'%epoch)

  model_ft.train()    # Initiate the model in training mode

  running_loss = 0
  correct = 0
  total = 0

  for data in tqdm(train_loader):

    inputs,labels=data[0].to(device),data[1].to(device)   # Loading the input tensors into CUDA GPU
    # Generate adversarial examples using FGSM
    inputs.requires_grad = True

    optimizer.zero_grad()
    outputs = model_ft(inputs)
    loss = criterion(outputs,labels)  # Calculating the loss
    loss.backward()                   # Back Propagation for calculaing gradients and adjusting weights

    # Create perturbations based on the sign of gradients
    inputs_grad = inputs.grad.data
    perturbed_inputs = torch.clamp(inputs + epsilon * torch.sign(inputs_grad), 0, 1)
    perturbed_inputs = perturbed_inputs.detach()

    # Perform forward pass with perturbed inputs
    outputs = model_ft(perturbed_inputs)
    loss = criterion(outputs, labels)
    loss.backward()

    optimizer.step()

    running_loss += loss.item()

    _, predicted = outputs.max(1)
    total += labels.size(0)
    correct += predicted.eq(labels).sum().item()




  train_loss = running_loss/len(train_loader)     # Calculating the mean of training loss
  accu = 100.*correct/total                       # Calculating the accuracy

  train_accu.append(accu)
  train_losses.append(train_loss)
  print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))

In [43]:
  # Validation
def eval_val(epoch):
    model_ft.eval()
    wrong_predictions = []
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
      for i, data in enumerate(val_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        # Get predicted labels
        _, predicted = torch.max(outputs.data, 1)

        # Find wrong predictions
        for j in range(len(predicted)):
          if predicted[j] != labels[j]:
              wrong_predictions.append({
                  'input': inputs[j],
                  'predicted_label': predicted[j].item(),
                  'true_label': labels[j].item()
              })
        # print(val_loss)
        val_loss += loss.item()
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()


    val_batch_loss = val_loss/len(val_loader)
    val_batch_accu = 100.*correct/total                       # Calculating the accuracy
    val_accu.append(val_batch_accu)
    print('Val Loss: %.3f| Val Accuracy: %.3f' %(val_batch_loss,val_batch_accu))

    print('Top 5 Wrong Predictions in Validation Set:')
    counter = Counter([prediction['true_label'] for prediction in wrong_predictions])
    top_wrong_predictions = counter.most_common(5)

    for true_label, count in top_wrong_predictions:
        print(f'True Label: {true_label}, Count: {count}')
        filtered_predictions = [prediction for prediction in wrong_predictions
                                if prediction['true_label'] == true_label][:5]
        for prediction in filtered_predictions:
            input_image = prediction['input']
            predicted_label = prediction['predicted_label']
            true_label = prediction['true_label']
            print(f'Predicted Label: {predicted_label}, True Label: {true_label}')


    # # Print details of wrong predictions
    # print('Wrong Predictions in Validation Set:')
    # for prediction in wrong_predictions:
    #     input_image = prediction['input']
    #     predicted_label = prediction['predicted_label']
    #     true_label = prediction['true_label']
    #     print(f'Predicted Label: {predicted_label}, True Label: {true_label}')
    #     # Additional code to visualize or process the input image if needed


In [ ]:
epochs = 20 # 20 run modify it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)


Epoch : 1


100%|██████████| 296/296 [04:13<00:00,  1.17it/s]


Train Loss: 1.367 | Accuracy: 67.816
Val Loss: 1.143| Val Accuracy: 70.823
Top 5 Wrong Predictions in Validation Set:
True Label: 148, Count: 14
Predicted Label: 55, True Label: 148
Predicted Label: 129, True Label: 148
Predicted Label: 149, True Label: 148
Predicted Label: 29, True Label: 148
Predicted Label: 189, True Label: 148
True Label: 108, Count: 13
Predicted Label: 40, True Label: 108
Predicted Label: 41, True Label: 108
Predicted Label: 96, True Label: 108
Predicted Label: 38, True Label: 108
Predicted Label: 138, True Label: 108
True Label: 4, Count: 10
Predicted Label: 162, True Label: 4
Predicted Label: 213, True Label: 4
Predicted Label: 39, True Label: 4
Predicted Label: 183, True Label: 4
Predicted Label: 194, True Label: 4
True Label: 3, Count: 10
Predicted Label: 40, True Label: 3
Predicted Label: 12, True Label: 3
Predicted Label: 22, True Label: 3
Predicted Label: 40, True Label: 3
Predicted Label: 5, True Label: 3
True Label: 58, Count: 8
Predicted Label: 49, True 

100%|██████████| 296/296 [04:13<00:00,  1.17it/s]


Train Loss: 1.287 | Accuracy: 68.885
Val Loss: 1.277| Val Accuracy: 68.777
Top 5 Wrong Predictions in Validation Set:
True Label: 148, Count: 16
Predicted Label: 194, True Label: 148
Predicted Label: 42, True Label: 148
Predicted Label: 245, True Label: 148
Predicted Label: 15, True Label: 148
Predicted Label: 122, True Label: 148
True Label: 108, Count: 12
Predicted Label: 136, True Label: 108
Predicted Label: 147, True Label: 108
Predicted Label: 139, True Label: 108
Predicted Label: 136, True Label: 108
Predicted Label: 35, True Label: 108
True Label: 129, Count: 10
Predicted Label: 56, True Label: 129
Predicted Label: 28, True Label: 129
Predicted Label: 8, True Label: 129
Predicted Label: 26, True Label: 129
Predicted Label: 40, True Label: 129
True Label: 123, Count: 9
Predicted Label: 244, True Label: 123
Predicted Label: 117, True Label: 123
Predicted Label: 23, True Label: 123
Predicted Label: 160, True Label: 123
Predicted Label: 197, True Label: 123
True Label: 82, Count: 9


...

100%|██████████| 296/296 [04:09<00:00,  1.18it/s]


Train Loss: 1.159 | Accuracy: 71.243
Val Loss: 1.312| Val Accuracy: 68.396
Top 5 Wrong Predictions in Validation Set:
True Label: 4, Count: 14
Predicted Label: 72, True Label: 4
Predicted Label: 161, True Label: 4
Predicted Label: 62, True Label: 4
Predicted Label: 176, True Label: 4
Predicted Label: 100, True Label: 4
True Label: 148, Count: 13
Predicted Label: 70, True Label: 148
Predicted Label: 29, True Label: 148
Predicted Label: 64, True Label: 148
Predicted Label: 129, True Label: 148
Predicted Label: 232, True Label: 148
True Label: 99, Count: 11
Predicted Label: 40, True Label: 99
Predicted Label: 128, True Label: 99
Predicted Label: 40, True Label: 99
Predicted Label: 234, True Label: 99
Predicted Label: 40, True Label: 99
True Label: 129, Count: 11
Predicted Label: 40, True Label: 129
Predicted Label: 40, True Label: 129
Predicted Label: 11, True Label: 129
Predicted Label: 232, True Label: 129
Predicted Label: 174, True Label: 129
True Label: 183, Count: 9
Predicted Label: 

100%|██████████| 296/296 [04:09<00:00,  1.19it/s]


Train Loss: 1.126 | Accuracy: 72.354
Val Loss: 1.427| Val Accuracy: 65.731
Top 5 Wrong Predictions in Validation Set:
True Label: 148, Count: 13
Predicted Label: 252, True Label: 148
Predicted Label: 31, True Label: 148
Predicted Label: 34, True Label: 148
Predicted Label: 234, True Label: 148
Predicted Label: 11, True Label: 148
True Label: 108, Count: 12
Predicted Label: 65, True Label: 108
Predicted Label: 36, True Label: 108
Predicted Label: 3, True Label: 108
Predicted Label: 220, True Label: 108
Predicted Label: 162, True Label: 108
True Label: 99, Count: 11
Predicted Label: 40, True Label: 99
Predicted Label: 217, True Label: 99
Predicted Label: 112, True Label: 99
Predicted Label: 119, True Label: 99
Predicted Label: 189, True Label: 99
True Label: 0, Count: 11
Predicted Label: 221, True Label: 0
Predicted Label: 40, True Label: 0
Predicted Label: 194, True Label: 0
Predicted Label: 180, True Label: 0
Predicted Label: 127, True Label: 0
True Label: 38, Count: 10
Predicted Label

 13%|█▎        | 38/296 [00:32<03:38,  1.18it/s]

In [ ]:
epochs = 12 # 20 run moidfy it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)

In [ ]:
epochs = 8 # 20 run modify it later on
for epoch in range(1, epochs+1):
  train(epoch)
  eval_val(epoch)

Validatation data sets as well - todo

Hyper Parameters = epcoh, learning rate,

In [ ]:
PATH = '/content/CalTech-256-GoogleNet_v23_1_adv.pth'
torch.save(model_ft.state_dict(), PATH)

In [ ]:
plt.plot(train_accu,'-o')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['Train'])
plt.show()

In [ ]:
plt.plot(train_losses,'-o')
plt.xlabel('epoch')
plt.ylabel('Loss')
plt.legend(['Train'])
plt.title('Train Loss')
plt.show()

In [ ]:
PATH='/content/CalTech-256-GoogleNet_v23_1_adv.pth'
model_ft.load_state_dict(torch.load(PATH))

In [ ]:
model_ft.eval()

In [ ]:
# Loading the train dataset
test_batch_size = 1
test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_batch_size)

In [ ]:
predictions = []
for i,data in enumerate(test_loader):
  input,_ = data
  output = model_ft(input.to(device))
  # print(output)
  pred = torch.max(output,dim=1)
  # print(pred)
  predictions.append(total_dataset.classes[pred.indices])

In [ ]:
# print([x[0].split("/",3)[3] for x in list(test_loader.dataset.imgs)])

In [ ]:
results = pd.DataFrame()
results['img_path'] = pd.Series([x[0].split("/",3)[3] for x in list(test_loader.dataset.imgs)])
results['label'] = pd.Series(predictions)

In [ ]:
results.to_csv("Group11_Kaggle2_Submission_v23_1_adv.csv")

In [ ]:
results.head(5)

In [ ]:
# results.head(5)

In [ ]:
from google.colab import files
# files.download('Group4_Pred_Submission_v75_lC_withques.csv')
files.download('Group11_Kaggle2_Submission_v23_1_adv.csv')
files.download('CalTech-256-GoogleNet_v23_1_adv.pth')

In [ ]:
# submit the file to kaggle
# !kaggle competitions submit classification-of-caltech-256-images -f Group11_Kaggle2_Submission_v4_adv.csv -m "Model"

#### Visualize the sample images of each class


### **Stage 2:** Build and train the CNN model using Keras/Pytorch (5 points)

You can train the CNN model and Pre-trained model and then compare the model performance on the kaggle testset


### Transfer learning

Transfer learning consists of taking features learned on one problem, and leveraging them on a new, similar problem.

A pre-trained model is a saved network that was previously trained on a large dataset, typically on a large-scale image-classification task.

The intuition behind transfer learning for image classification is that if a model is trained on a large and general enough dataset, this model will effectively serve as a generic model of the visual world. You can then take advantage of these learned feature maps without having to start from scratch by training a large model on a large dataset.



#### Use the pre-trained models

* Load the pre-trained model
* Train and evaluate the images

In [ ]:
# YOUR CODE HERE

###   **Stage 3**: Evaluate the Model and get model predictions on the Kaggle testset (2 Points)









In [ ]:
# YOUR CODE HERE

### Report Analysis

- Compare the accuracies for the Pre-trained vs CNN models
- What process was followed to tune the hyperparameters?
- Plot the confusion matrix in terms of the misclassifications